## Setup

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00


In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00


In [ ]:
pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 8.1 MB/s eta 0:00:00


In [ ]:
# libraries
import pandas as pd
import torch
import json
import os

from getpass import getpass
from langchain import HuggingFaceHub

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import SequentialChain

from transformers import AutoTokenizer, pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM # for Flan-T5 models
from transformers import LlamaTokenizer, LlamaForCausalLM ## for LLaMA model

set_seed(42)

In [ ]:
def load_jsonl(path):
    lines = []
    with open(path) as file:
        lines = file.read().splitlines()
    jsons = [json.loads(line) for line in lines]
    df = pd.DataFrame(jsons)

    return df

In [ ]:
aNLI_test = load_jsonl('test.jsonl')
label_test = pd.read_csv('test-labels.lst', header=None, names=['label'])
aNLI_test = aNLI_test.join(label_test)
print('test data length:', len(aNLI_test))



#aNLI_train = load_jsonl('train.jsonl')
#label_train = pd.read_csv('train-labels.lst', header=None, names=['label'])
#aNLI_train = aNLI_train.join(label_test)

aNLI_train = pd.read_csv('aNLI_train.csv')

pd.set_option('display.max_colwidth', None)
aNLI_test.head(2)

test data length: 3059


,story_id,obs1,obs2,hyp1,hyp2,label
0,87aa0983-9b84-48b1-86ff-160b1567487c-1,Jane was a professor teaching piano to students.,Jane spent the morning sipping coffee and reading a book.,Two of Jane's students were early for their lessons.,None of Jane's students had a lesson that day.,2
1,dfc8584e-13fe-4e26-bdf6-2485e90ef29d-1,Nate had the summer off before college.,Nate's last summer before college was a total blast!,Nate spent the summer traveling and partying.,Nate decided to spend the entire summer working in the Mines.,1


In [ ]:
# logging to huggingface
HUGGINGFACEHUB_API_TOKEN = getpass()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


In [ ]:
#HuggingFace models setup
temp = 0.1
length = 2000
tokens = 64

# FLAN-T5 model - 11.3B parameters; xl - 3B
llm_t5 = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"max_new_tokens": tokens})#, "max_length": length})

# Falcon model - or just without instruct??????? try!!!!!!!!
llm_falcon = HuggingFaceHub(repo_id="tiiuae/falcon-7b", model_kwargs={"max_new_tokens": tokens})#, "max_length": length})
llm_falcon_inst = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"max_new_tokens": tokens})#, "max_length": length})


In [ ]:
"""
pipe = pipeline(
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl"),
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl"),
    task = "text2text-generation", # text-generation for GPT and LLaMA
    torch_dtype=torch.float16,
    max_new_tokens = 64,
)

"""
pipe = pipeline(
    model = LlamaForCausalLM.from_pretrained('openlm-research/open_llama_3b'),
    tokenizer = LlamaTokenizer.from_pretrained('openlm-research/open_llama_3b'),
    task = "text-generation",
    torch_dtype=torch.float16,
    max_new_tokens = 64
)
"""
pipe = pipeline(
    tokenizer = AutoTokenizer.from_pretrained('gpt2-xl'),
    model = AutoModelForCausalLM.from_pretrained('gpt2-xl'),
    task = "text-generation",
    torch_dtype=torch.float16,
    max_new_tokens = 64,
)
"""

llm = HuggingFacePipeline(pipeline=pipe)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


## Zero shot

In [ ]:
# based on paper "Are Large Language Models Really Good Logical Reasoners?"
template_zero_shot = """Given a context, the abductive reasoning task is to choose the more likely explanation from a given pair of hypotheses choices. And give simple explanations.
The context is: {O1} {O2}
The hypothesis choices are: A. {H1} B. {H2}."""

prompt_zero_shot = PromptTemplate.from_template(template_zero_shot)

In [ ]:
# based on paper "Are Large Language Models Really Good Logical Reasoners?"
template_zero_shot = """Given a context, the abductive reasoning task is to choose the more likely explanation from a given pair of hypotheses choices. And give simple explanations.
The context is: {O1} {O2}
The hypothesis choices are: A. {H1} B. {H2}"""

prompt_zero_shot = PromptTemplate(input_variables=["O1", "O2", "H1", "H2"], template=template_zero_shot)

############################################

template_zero_shot_different1 = """You will get a context and a pair of hypotheses. The task is to select the most proable hypothesis given the context and give a basic explanation.
Context: {O1} {O2}
Hypotheses: A. {H1} B. {H2}
Answer: """

prompt_zero_shot_different1 = PromptTemplate.from_template(template_zero_shot_different1)

###########################################

template_zero_shot_different2 = """Select the more probable explanation of given context from a pair of hypothesis choices.
Context: {O1} {O2}
Hypothesis choices: A. {H1} B. {H2}
Answer: """

prompt_zero_shot_different2 = PromptTemplate.from_template(template_zero_shot_different2)

###########################################

prompt_zero_shot_different3 = """ First context C1 happened, then hypothesis happened, then context C2 happened.
The task is to choose one hypothesis from a given pair of hypotheses. The chosen hypothesis is coherent with the contexts and does not contradict them.
If both hypothesis are correct, the more likely one is chosen.
Context: C1. {O1} C2. {O2}
Hypothesis choices: A. {H1} B. {H2}
Answer: """

prompt_zero_shot_different3 = PromptTemplate.from_template(prompt_zero_shot_different3)


In [ ]:
template_reformulated = """The reasoning task is to choose the more consistent and more likely scenario from a given pair of scenarios.
Scenario A: {O1} {H1} {O2}
Scenario B: {O1} {H2} {O2}
Answer: """

prompt_reformulated = PromptTemplate.from_template(template_reformulated)


template_reformulated_CoS = """The reasoning task is to choose the more consistent and more likely scenario from a given pair of scenarios.
O1 is: {O1}, O2 is: {O2}, H1 is: {H1}, H2 is: {H2}
Scenario A is: O1 H1 O2
Scenario B is: O1 H2 O2
Answer: """

prompt_reformulated_CoS = PromptTemplate.from_template(template_reformulated_CoS)

###############################################

similarity_template_old = """Select a hypothesis that si more similar to the context. Context: {O1} {O2}
Hypothesis choices: A. {H1} B. {H2} Answer:"""

similarity_template2 = """Context: {O1} {O2} Hypotheses A: {H1} Hypothesis B: {H2}
Which of the two Hypothesis si more similar to the Context, Hypothesis A or Hypothesis B? """

similarity_template3 = """Context: {O1} {O2} Text A: {H1} Text B: {H2}
Which Text (A or B) si more similar to the Context?"""

similarity_prompt = PromptTemplate.from_template(similarity_template3)


## CoT

In [ ]:
# aNLI_train[(aNLI_train.story_id == '2018e090-9dca-4124-b80f-6ce21a7e9521-1') & (aNLI_train.hyp2=='The bakery had no cupcakes left.')]
# aNLI_train[(aNLI_train.story_id == 'a62e9212-e75b-4078-ae68-f56a1164dc702') & (aNLI_train.hyp1=='We ate soup every night.')]
# aNLI_train[(aNLI_train.story_id == 'c96c962e-7fe3-4e5b-b40a-fb3a9364f41d-1') & (aNLI_train.hyp2=='Everyone was given soup.')]
# aNLI_train[(aNLI_train.story_id == 'd8dbfb94-0ad0-4b0c-a55c-173dc1a91898-1') & (aNLI_train.hyp1=='Jake saw a tree.')]

template_CoT = """Select the more probable explanation of given context from a pair of hypothesis choices and give an explanation.
Next, I will give you 4 examples.

Context: Jenny wanted cupcakes. She bought them all.
Hypothesis choices: A. The bakery only had three cupcakes left. B. The bakery had no cupcakes left.
Answer: Hypothesis B. contradicts the context because Jenny can not buy cupcakes as there are no cupcakes left.
The correct hypothesis is A.

Context: Jake was at the park. He chased after it but never caught it.
Hypothesis choices: A. Jake saw a tree. B. Jake saw a squirrel.
Answer: Hypothesis A. contradicts the context because trees stand in one place and you do not chase them.
The correct hypothesis is B.

Context: When I was younger, I went to summer camp. My favorite card game was poker.
Hypothesis choices: A. We ate soup every night. B. We played cards a lot.
Answer: No hypothesis contradicts the context. Hypothesis B. relates better to the context because it is about card game.
The correct hypothesis is B.

Context: It was a swelteringly hot day. They all loved the cool treat!
Hypothesis choices: A. Everyone was given ice cream. B. Everyone was given soup.
Answer: No hypothesis contradicts the context. Hypothesis A. relates better to the context because ice cream is a cool treat.
The correct hypothesis is A.

Context: {O1} {O2}
Hypothesis choices: A. {H1} B. {H2}
Answer:
"""

CoT_prompt = PromptTemplate( input_variables=["O1", "O2", "H1", "H2"], template=template_CoT)



In [ ]:
# aNLI_train[(aNLI_train.story_id == '2018e090-9dca-4124-b80f-6ce21a7e9521-1') & (aNLI_train.hyp2=='The bakery had no cupcakes left.')]
# aNLI_train[(aNLI_train.story_id == 'a62e9212-e75b-4078-ae68-f56a1164dc702') & (aNLI_train.hyp1=='We ate soup every night.')]
# aNLI_train[(aNLI_train.story_id == 'c96c962e-7fe3-4e5b-b40a-fb3a9364f41d-1') & (aNLI_train.hyp2=='Everyone was given soup.')]
# aNLI_train[(aNLI_train.story_id == 'd8dbfb94-0ad0-4b0c-a55c-173dc1a91898-1') & (aNLI_train.hyp1=='Jake saw a tree.')]

template_CoT2 = """First context C1 happened, then hypothesis happened, then context C2 happened.
The task is to choose one hypothesis from a given pair of hypotheses. The chosen hypothesis is coherent with the contexts
and does not contradict them. If both hypothesis are correct, the more likely one is chosen.
Next, I will give you 4 examples.

Context: Jenny wanted cupcakes. She bought them all.
Hypothesis choices: A. The bakery only had three cupcakes left. B. The bakery had no cupcakes left.
Answer: Does one of the hypothesis contradict the context? Yes, hypothesis B because Jenny can not buy cupcakes as there are no cupcakes left.
The correct hypothesis is A.

Context: When I was younger, I went to summer camp. My favorite card game was poker.
Hypothesis choices: A. We ate soup every night. B. We played cards a lot.
Answer: Does one of the hypothesis contradict the context? No. Hypothesis B relates better to the context because it is about card game.
The correct hypothesis is B.

Context: Jake was at the park. He chased after it but never caught it.
Hypothesis choices: A. Jake saw a tree. B. Jake saw a squirrel.
Answer: Does one of the hypothesis contradict the context? Yes, hypothesis A because trees stand in one place and you do not chase them.
The correct hypothesis is B.

Context: It was a swelteringly hot day. They all loved the cool treat!
Hypothesis choices: A. Everyone was given ice cream. B. Everyone was given soup.
Answer: Does one of the hypothesis contradict the context? No. Hypothesis A relates better to the context because ice cream is a cold.
The correct hypothesis is A.

Context: {O1} {O2}
Hypothesis choices: A. {H1} B. {H2}
Answer:
"""

CoT_prompt2 = PromptTemplate( input_variables=["O1", "O2", "H1", "H2"], template=template_CoT2)



In [ ]:
# first one is made up
# 7f0ff4e8-3c77-41a5-8107-916dd66cb22d2

template_CoT3 = """Select the more probable explanation of given context from a pair of hypothesis choices and give an explanation.
Next, I will give you 4 examples.

Context: Alice has a ball. Alice does not have the ball.
Hypothesis choices: A. Alice gave the ball to Bill. B. Alice kept the ball.
Answer: Does one of the hypothesis contradict the context? Yes, hypothesis B contradicts context because Alice kept the ball so she has it.
That Alice gave the ball away explains why she does not have it. The context is better explained by hypothesis A.

Context: Neil was traveling in Vietnam. He had a great time on his educational trip.
Hypothesis choices: A. Neil decided to teach new things to local people. B. It was a college class trip.
Answer: Does one of the hypothesis contradict the context? No. It is  more likely that the educational trip was a college class trip.
The context is better explained by hypothesis B.

Context: I was walking to the mall. I almost cracked my phone.
Hypothesis choices: A. I tripped on a stick. B. On the way, I realized I forgot my phone.
Answer: Does one of the hypothesis contradict the context? Yes, hypothesis B contradicts the context because I would not break phone that I do not have because I forgot it.
Tripping on a stick better explaines why the phone is cracked. The context is better explained by hypothesis B.

Context: When I was five, I moved. I felt less lonely.
Hypothesis choices: A. I made a lot of new friends after that. B. Many kids in the old place wanted to be my friends.
Answer: Does one of the hypothesis contradict the context? No. It is more likely that making new friends in the new home makes you feel less lonely.
The context is better explained by hypothesis A.

Context: {O1} {O2}
Hypothesis choices: A. {H1} B. {H2}
Answer:
"""

CoT_prompt3 = PromptTemplate( input_variables=["O1", "O2", "H1", "H2"], template=template_CoT3)



In [ ]:
# Zero shot CoT
template_zero_CoT = """Given a context, the abductive reasoning task is to choose the more likely explanation from a given pair of hypotheses choices. And give simple explanations.
The context is: {O1} {O2}
The hypothesis choices are: A. {H1} B. {H2}
Let’s think step by step. """

prompt_zero_CoT = PromptTemplate.from_template(template_zero_CoT)

## Run

In [ ]:
def give_predictions_zero_shot(model, prompt, samples, save):
    answers = []
    story_id = []
    c = 1

    llm_chain = LLMChain(prompt=prompt, llm=model)

    for n in samples.index:
        obs1 = samples.obs1[n]
        obs2 = samples.obs2[n]
        hyp1 = samples.hyp1[n]
        hyp2 = samples.hyp2[n]

        answers.append(llm_chain.run({"O1":obs1,"O2":obs2,"H1":hyp1,"H2":hyp2}))
        story_id.append(samples.story_id[n])

        if c%10 == 0:
            pd.DataFrame({'story_id':story_id, 'answer':answers}).to_csv(save, index=False)
            print(c)
        c += 1

    pd.DataFrame({'story_id':story_id, 'answer':answers}).to_csv(save, index=False)
    print('samples answered: ', len(answers))
    return answers

In [ ]:
%%time
llama = give_predictions_zero_shot(llm, CoT_prompt2, aNLI_test[:600], 'anli_llama_CoT2_task0.csv')


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
samples answered:  600
CPU times: user 1d 5h 57min 58s, sys: 9min 54s, total: 1d 6h 7min 52s
Wall time: 7h 32min 2s


## Few-shot


In [ ]:
# based on paper "Are Large Language Models Really Good Logical Reasoners?"
template_example = """The context is: {ex_O1} {ex_O2}
The hypothesis choice are: A. {ex_H1} B. {ex_H2}
The correct choice is: {ex_label}"""

few_shot_prompt = PromptTemplate( input_variables=["ex_O1", "ex_O2", "ex_H1", "ex_H2","ex_label"], template=template_example)

In [ ]:
template_example_reform = """Scenario A: {ex_O1} {ex_H1} {ex_O2} Scenario B: {ex_O1} {ex_H2} {ex_O2}
The correct scenario is {ex_label}"""

few_shot_reform_prompt = PromptTemplate.from_template(template_example_reform)


template_example_reform_SoT = """O1 is: {ex_O1}, O2 is: {ex_O2}, H1 is: {ex_H1}, H2 is: {ex_H2}
Scenario A is: O1 H1 O2 Scenario B is: O1 H2 O2
The correct scenario is {ex_label}"""

few_shot_reform_prompt_SoT = PromptTemplate.from_template(template_example_reform_SoT)


In [ ]:
def give_predictions_few_shot(model, shot, prompt, samples, examples, save, scenario, CoS, hypothesis):
    answers = []
    story_id = []
    c = 1

    if scenario == False:
        # original prompt
        pf = "Given a context, the abductive reasoning task is to choose the more likely explanation from a given pair of hypotheses choices. And give simple explanations. Next, I will give you {k} example(s) for test."
    else:
        # scenario prompt:
        pf = "The reasoning task is to choose the more consistent and more likely scenario from a given pair of scenarios. Next, I will give you {k} example(s) for test."

    if scenario == False and CoS == False:
        # original prompt
        sf = """Next, I will give you an example for test. The context is: {O1} {O2} The hypothesis choices are: A. {H1} B. {H2}"""
    elif scenario == True and CoS == False:
        # scenario prompt
        sf = """Next, I will give you an example for test. Scenario A: {O1} {H1} {O2} Scenario B: {O1} {H2} {O2}"""
    else:
        # scenario with CoS
        sf = """Next, I will give you an example for test. O1 is: {O1}, O2 is: {O2}, H1 is: {H1}, H2 is: {H2} Scenario A is: O1 H1 O2 Scenario B is: O1 H2 O2"""

    for n in samples.index:
        obs1 = samples.obs1[n]
        obs2 = samples.obs2[n]
        hyp1 = samples.hyp1[n]
        hyp2 = samples.hyp2[n]

        created_examples = []
        for i in range(shot):
            ex = examples.sample(1)
            if ex.label.values[0] == 1:
                label = 'A.'
                # to add the explicit text of solution
                if hypothesis == True:
                    label = label + ' ' + ex.hyp1.values[0]
            elif ex.label.values[0] == 2:
                label = 'B.'
                if hypothesis == True:
                    label = label + ' ' + ex.hyp2.values[0]
            else:
                print('ERROR')
            created_examples.append({"ex_O1":ex.obs1.values[0], "ex_O2":ex.obs2.values[0], "ex_H1":ex.hyp1.values[0],
                "ex_H2":ex.hyp2.values[0],"ex_label":label})

        prompt_few_shot = FewShotPromptTemplate(examples=created_examples, example_prompt=prompt, prefix=pf, suffix=sf,
                                                input_variables=["k","O1", "O2", "H1", "H2"])

        llm_chain = LLMChain(prompt=prompt_few_shot, llm=model)

        answers.append(llm_chain.run({"k":shot,"O1":obs1,"O2":obs2,"H1":hyp1,"H2":hyp2}))
        story_id.append(samples.story_id[n])

        if c%10 == 0:
            pd.DataFrame({'story_id':story_id, 'answer':answers}).to_csv(save, index=False)
            print(c)
        c += 1

    pd.DataFrame({'story_id':story_id, 'answer':answers}).to_csv(save, index=False)
    print('samples answered: ', len(answers))
    return answers

In [ ]:
%%time
predictions = give_predictions_few_shot(llm, 3, few_shot_reform_prompt_SoT, aNLI_test[300:600], aNLI_train,
                                        'anli_llama_few_shot3_reform_SoT.csv',
                                        scenario=True, CoS=True, hypothesis=False)


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
samples answered:  300
CPU times: user 11h 57min 11s, sys: 13min 50s, total: 12h 11min 2s
Wall time: 3h 3min 10s


## Model leakage

In [ ]:
prompt_train = aNLI_train.obs1[0] + ' ' + aNLI_train.obs2[0]
print(prompt_train)
print('answer: ', llm_t5(prompt_train))
print('hypotheses: ', aNLI_train.hyp1[0], aNLI_train.hyp2[0])
print('id: ', llm_t5(aNLI_train.story_id[0]))

In [ ]:
sample = aNLI_train.sample(1)
prompt_test = sample.obs1.tolist()[0] + ' ' + sample.obs2.tolist()[0]
print(prompt_test)
print('answer: ', llm_t5(prompt_test))
print('hypotheses: ', sample.hyp1.tolist()[0], sample.hyp2.tolist()[0])
print('id: ', llm_t5(sample.story_id.tolist()[0]))